In [2]:
from time import time
import numpy as np
import matplotlib.pyplot as plt

## Zadanie 1

In [141]:
def gauss_jordan(A, B):
    n = A.shape[0]
    AB = np.hstack((A, B))

    # SCALING
    for i in range(n):
        AB[i] /= np.max(np.abs(A[i]))

    for i in range(n):
        # pivot = i + np.abs(A[i:,i]).argmax()
        # A[[i, pivot]] = A[[pivot, i]]

        AB[i] /= AB[i, i]

        for j in range(n):
            if i != j:
                AB[j] -= AB[j, i] * AB[i]

    return AB[:,n]

In [72]:
def random_a_b_integers(n, _min=-1000, _max=1000):
    a = np.random.randint(_min, _max, n * n).astype(np.float64).reshape(n, n)
    b = np.random.randint(_min, _max, n).astype(np.float64).reshape(n, 1)
    return a, b

Gauss-Jordan:  1.1938092708587646 sekund
np.linalg.solve:  0.001994609832763672 sekund
Gauss-Jordan:  1.5179386138916016 sekund
np.linalg.solve:  0.003991842269897461 sekund
Gauss-Jordan:  2.2200610637664795 sekund
np.linalg.solve:  0.004987239837646484 sekund
Gauss-Jordan:  3.064847946166992 sekund
np.linalg.solve:  0.0059435367584228516 sekund
Gauss-Jordan:  4.086074113845825 sekund
np.linalg.solve:  0.011968374252319336 sekund


In [131]:
def random_a_b_float(n, _min=-100000, _max=100000):
    a = np.random.uniform(_min, _max, [n, n])
    b = np.random.uniform(_min, _max, [n, 1])
    return a, b

In [143]:
a, b = random_a_b_float(500)
gauss_jordan(a, b).reshape(500, 1) - np.linalg.solve(a, b)

array([[ 4.04176692e-13],
       [ 2.12674323e-12],
       [ 1.54920521e-12],
       [-7.87377108e-13],
       [ 4.90707475e-12],
       [-1.69936808e-12],
       [ 5.11191089e-12],
       [-2.21656027e-13],
       [-3.11421375e-12],
       [ 8.29336599e-13],
       [-4.29023483e-12],
       [-1.38467016e-12],
       [-1.64446234e-12],
       [-4.18332036e-13],
       [ 1.08968390e-12],
       [ 6.17284002e-13],
       [-1.07047704e-12],
       [-1.39456805e-12],
       [-6.95443703e-13],
       [-5.55999691e-13],
       [-6.06714678e-12],
       [-1.27098332e-12],
       [-8.80673312e-12],
       [ 6.26743102e-12],
       [ 1.69753100e-12],
       [ 3.55049323e-13],
       [ 4.34322023e-12],
       [-3.97415434e-12],
       [ 4.75719464e-12],
       [ 1.38133949e-12],
       [ 1.98108197e-12],
       [-2.29372077e-12],
       [ 3.49720253e-13],
       [ 1.62880820e-12],
       [-1.87938554e-12],
       [-1.15645514e-12],
       [ 1.66267000e-12],
       [ 3.20898863e-12],
       [-1.0

In [127]:
np.linalg.solve(a, b).shape

(20, 1)

In [ ]:
a = np.array([2, 2, 6, 2, 1, 7, -2, -6, -7], dtype=np.float64).reshape(3, 3)
b = np.array([4, 6, -1], dtype=np.float64).reshape(3, 1)
gauss_jordan(a, b)
a = np.random.randint(-1000, 1000, 500 * 500).astype(np.float64).reshape(500, 500)
b = np.random.randint(-1000, 1000, 500).astype(np.float64).reshape(500, 1)
gauss_jordan(a, b)
np.linalg.solve(a, b)

In [ ]:
for x in range(500, 1000, 100):
    a, b = random_a_b_integers(x)

    start = time()
    gauss_jordan(a, b)
    print("Gauss-Jordan: ", time() - start, "sekund")

    start = time()
    np.linalg.solve(a, b)
    print("np.linalg.solve: ", time() - start, "sekund")